# Speech to text baseline

In [ ]:
!pip install torchaudio
!pip install transformers
!pip install datasets
!pip install lang_trans
!pip install arabic_reshaper
!pip install python-bidi
!pip install pydub
!pip install tnkeeh
!pip install soundfile

In [ ]:
import os
import torch
import torchaudio
import numpy as np
import pandas as pd
import tnkeeh as tn
import seaborn as sns
import soundfile as sf
import arabic_reshaper
from pydub import AudioSegment
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from datasets import load_dataset, Dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
% matplotlib inline

# Connect to drive and load dataset

In [ ]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/All_poems.zip .

In [ ]:
if os.path.exists('dataset'):
  if len(os.listdir('dataset')) == 0:
    os.system('unzip All_poems.zip -d dataset')
else:
  os.system('unzip All_poems.zip -d dataset')

In [ ]:
!mkdir -p dataset_wav

In [ ]:
metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/جدول جميع الأبيات .xlsx'
dataset_folder = 'dataset'
dataset_wav_folder = 'dataset_wav' 

# Read and preprocess the dataframe

In [ ]:
metadata = pd.read_excel(metadata_path)
metadata['Bahr'] = metadata['Bahr'].apply(lambda bahr: bahr.strip() if isinstance(bahr,str) else bahr)
metadata['Clean Script'] = metadata['Script']
#TODO: use pyarabic as tnkeeh strips newlines
metadata = tn.clean_data_frame(metadata, 'Clean Script',remove_diacritics = True, remove_tatweel = True)
metadata['File Exist'] = metadata['Utterance name'].apply(lambda file_name: os.path.isfile(f'{dataset_folder}/{file_name}'))
metadata = metadata[metadata['File Exist'] == True]
metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Further info on Bahr,Name of document,Clean Script,File Exist
0,1,P-1.aac,شتَّ شَعبُ الحيِّ بعدَ التئامْ وشجاكَ الر...,المديد,ديوان الطرماح ص227,1.aac,1.aac,NaN,بحر المديد,شت شعب الحي بعد التئام وشجاك الربع ربع المقام,True
1,2,P-2.aac,حَسَرَتْ عَنْهُ الرِّيَاحُ فَأَبْدَتْ من...,المديد,ديوان الطرماح ص227,2.aac,2.aac,NaN,NaN,حسرت عنه الرياح فأبدت منتأى كالقرو رهن انثلام,True
2,3,P-3.aac,وخصيفَ اللَّونِ جادَتْ بهِ مَرْخَةٌ مِنْ ...,المديد,ديوان الطرماح ص227,3.aac,3.aac,NaN,NaN,وخصيف اللون جادت به مرخة من مخدج أو تمام,True
3,4,P-4.aac,بَيْنَ أظْآرٍ بِمَظْلُومَةٍ كَسَرَاةِ الس...,المديد,ديوان الطرماح ص227,4.aac,4.aac,NaN,NaN,بين أظآر بمظلومة كسراة الساق ساق الحمام,True
4,5,P-5.aac,مَنْزِلًا كَانَ لَنَا مَرَّةً وطنًا نحتلّ...,المديد,ديوان الطرماح ص227,5.aac,5.aac,NaN,NaN,منزلا كان لنا مرة وطنا نحتله كل عام,True
...,...,...,...,...,...,...,...,...,...,...,...
3730,3731,P-3731.mp3,وَلِّ السَلاطينَ مَن تَوَلّاها وَاِلجَأ إ...,المنسرح,-,45.mp3,45.mp3,NaN,NaN,ول السلاطين من تولاها والجأ إليه تكن حدياها,True
3731,3732,P-3732.mp3,وَلا تَغُرَّنَّكَ الإِمارَةُ في غَيرِ أَم...,المنسرح,-,46.mp3,46.mp3,NaN,NaN,ولا تغرنك الإمارة في غير أمير وإن بها باهى,True
3732,3733,P-3733.mp3,فَإِنَّما المَلكُ رَبُّ مَملَكَةٍ قَد فَغ...,المنسرح,-,47.mp3,47.mp3,NaN,NaN,فإنما الملك رب مملكة قد فغم الخافقين سرياها,True
3733,3734,P-3734.mp3,مُبتَسِمٌ وَالوُجوهُ عابِسَةٌ سِلمُ العِد...,المنسرح,-,48.mp3,48.mp3,NaN,NaN,مبتسم والوجوه عابسة سلم العدى عنده كهيجاها,True


# convert all non wav files to wav and get sampling rates

In [ ]:
sample_rates = set()
for file_path in metadata['Utterance name']:
  complete_path = f'{dataset_folder}/{file_path}'
  audio = AudioSegment.from_file(complete_path)
  sample_rates.add(audio.frame_rate)
  audio.export(f'{dataset_wav_folder}/{file_path}', format='wav')
sample_rates

KeyboardInterrupt: ignored

# Preparing dataset with speech arrays

In [ ]:
dataset = Dataset.from_pandas(metadata)
dataset

Dataset({
    features: ['No.', 'Utterance name', 'Script', 'Bahr', 'Source', ' Name when extacted', 'Original name in document', 'Further info on Bahr', 'Name of document', 'Clean Script', 'File Exist', '__index_level_0__'],
    num_rows: 3717
})

In [ ]:
resamplers = {  # The dataset contains all the uncommented sample rates
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    # 32000: torchaudio.transforms.Resample(32000, 16000),
}

In [ ]:
def prepare_example(example):
  complete_path = f'{dataset_wav_folder}/{example["Utterance name"]}'
  speech, sampling_rate = sf.read(complete_path)
  speech = torch.Tensor(speech)
  example["speech"] = resamplers[sampling_rate](speech).squeeze().numpy()
  return example

In [53]:
dataset1 = Dataset.from_dict(dataset[:20])

In [54]:
dataset1 = dataset1.map(prepare_example)
processor = Wav2Vec2Processor.from_pretrained("elgeish/wav2vec2-large-xlsr-53-arabic")
model = Wav2Vec2ForCTC.from_pretrained("elgeish/wav2vec2-large-xlsr-53-arabic").eval()

  0%|          | 0/20 [00:00<?, ?ex/s]

In [55]:
def predict(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        predicted = torch.argmax(model(inputs.input_values).logits, dim=-1)
    predicted[predicted == -100] = processor.tokenizer.pad_token_id  # see fine-tuning script
    batch["predicted"] = processor.tokenizer.batch_decode(predicted)
    return batch

In [56]:
dataset1 = dataset1.map(predict, batched=True, batch_size=1)

  0%|          | 0/20 [00:00<?, ?ba/s]

In [57]:
for reference, predicted,source,file_path in zip(dataset1["Script"], dataset1["predicted"],dataset1["Source"],dataset1['Utterance name']):
  print('diwan',source)
  print("reference:", reference)
  print("predicted:", buckwalter.untrans(predicted))
  print('file path:',file_path)
  print("--")

diwan ديوان الطرماح ص227  
reference: شتَّ شَعبُ الحيِّ بعدَ التئامْ      وشجاكَ الرَّبعُ رَبعُ المَقامْ
predicted: شَتَّ شَعْبُ الْحَيِّ بَعْدَ الْتِئَامِوَشَجَاكَ الرَّبْعُ رَبْعُ الْمَقَامِ
file path: P-1.aac
--
diwan ديوان الطرماح ص227
reference:  حَسَرَتْ عَنْهُ الرِّيَاحُ فَأَبْدَتْ      منتأًى كالقَرْوِ رَهْنَ انثِلامْ
predicted: حَسَرَتْ عَنْهُ الرِّيَاحُ فَأَبدَّتْ مُنْتَأً كَالْقَرْ رَهْنَا ثِلَامِ
file path: P-2.aac
--
diwan ديوان الطرماح ص227
reference: وخصيفَ اللَّونِ جادَتْ بهِ      مَرْخَةٌ مِنْ مُخْدَجٍ أَوْ تَمَامْ
predicted: وَخَصِيفَ اللَّوْنِ جَادَتْ بِهِ مَرْخَةٌ مِنْ مُخْدَجٍ أَوْ تَمَامٍ
file path: P-3.aac
--
diwan ديوان الطرماح ص227
reference: بَيْنَ أظْآرٍ بِمَظْلُومَةٍ      كَسَرَاةِ السَّاقِ سَاقِ الحَمَامْ
predicted: بين أضآار بمضلومةكسرات الساق ساق الحمام
file path: P-4.aac
--
diwan ديوان الطرماح ص227
reference: مَنْزِلًا كَانَ لَنَا مَرَّةً      وطنًا نحتلُّهُ كلَّ عامْ 
predicted: منزلا كان لنا مرّةً وطنًا نحتلّه كل عام
file path: P-5.aac
--
diwan ديوان ا